In [ ]:
import numpy as np
import pandas as pd
import scipy
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from scipy.signal import resample
from tensorflow.keras.optimizers import Adam
import os
import zipfile

In [ ]:
"""Mounting Google Drive"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
""" Select a Type of Data to be Generated"""

#Type can be selected as one of the follows:

#'MI', 'nback', 'wg', 'dsr'
type = 'MI'

In [ ]:
class ScaleLayer(keras.layers.Layer):
    def __init__(self, min_value, max_value, **kwargs):
        super(ScaleLayer, self).__init__(**kwargs)
        self.min_value = min_value
        self.max_value = max_value

    def call(self, inputs):
        return (inputs + 1.0) * 0.5 * (self.max_value - self.min_value) + self.min_value

In [ ]:
def generate_synthetic_data(generator, discriminator, num_samples, latent_dim):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    synthetic_data = generator.predict(noise)

    # Generate corresponding synthetic labels using the trained discriminator
    synthetic_labels = discriminator.predict(synthetic_data)

    return synthetic_data, synthetic_labels

In [ ]:
""" This function is an inverse of scaling we did earlier """

def NIRS_ReScaling(nirs_data, min_eeg, max_eeg, min_nirs, max_nirs):
  nirs_data = ((nirs_data - min_eeg) / (max_eeg - min_eeg)) * (max_nirs - min_nirs) + min_nirs
  return nirs_data

In [ ]:
def DownSampleNIRS(nirs_data, target_length):

  resampled_nirs_data = np.zeros((nirs_data.shape[0], target_length, nirs_data.shape[2]))

  for i in range(nirs_data.shape[0]):
      for j in range(nirs_data.shape[2]):
          resampled_nirs_data[i, :, j] = resample(nirs_data[i, :, j], target_length)

  return resampled_nirs_data

In [ ]:
""" Reading the corresponding Generators"""

custom_objects = {'ScaleLayer': ScaleLayer}

if type == 'MI':

  #Reading the Corresponding generator and discriminator
  generator = load_model("/content/drive/MyDrive/generator.h5", custom_objects=custom_objects)
  discriminator = load_model("/content/drive/MyDrive/discriminator.h5", custom_objects=custom_objects)

  #These values below have been taken from the original data
  min_eeg, max_eeg, min_nirs, max_nirs = -3280.4554707320476, 5057.0887589945505, -0.007749180583169982, 1.655906071960631

  #Labels
  labels = [16, 32]

  #Generating Data
  num_samples = 1000
  latent_dim = 100
  synthetic_data, synthetic_labels = generate_synthetic_data(generator, discriminator, num_samples, latent_dim)

  #Extracting EEG Data
  eeg_data = synthetic_data[:,:,0:32]

  #Extracting NIRS data
  nirs_data = synthetic_data[:,:,32:]
  nirs_data = NIRS_ReScaling(nirs_data, min_eeg, max_eeg, min_nirs, max_nirs)

  #Resampling as per original dimensions
  nirs_data = DownSampleNIRS(nirs_data, 270)      # 270 was the shape of original nirs data for MI (10 Hz)

  #Transforming Synthetic Labels based as 16 and 32
  synthetic_labels = np.round(synthetic_labels)
  synthetic_labels[synthetic_labels == 0] = 16
  synthetic_labels[synthetic_labels == 1] = 32
  synthetic_labels = np.concatenate(synthetic_labels, axis=0)

32/32 [==============================] - 25s 745ms/step


In [ ]:
if type == 'dsr':

  #Reading the Corresponding generator and discriminator
  generator = load_model("/content/drive/MyDrive/generator_dsr.h5", custom_objects=custom_objects)
  discriminator = load_model("/content/drive/MyDrive/discriminator_dsr.h5", custom_objects=custom_objects)

  #These values below have been taken from the original data
  min_eeg, max_eeg, min_nirs, max_nirs = -2354.830184279685, 3283.839866302977, -0.08963729193757349, 0.14012275848719927

  #Generating Data
  num_samples = 1000
  latent_dim = 100
  synthetic_data, synthetic_labels = generate_synthetic_data(generator, discriminator, num_samples, latent_dim)

  #Extracting EEG Data
  eeg_data = synthetic_data[:,:,0:30]

  #Extracting NIRS data
  nirs_data = synthetic_data[:,:,30:]
  nirs_data = NIRS_ReScaling(nirs_data, min_eeg, max_eeg, min_nirs, max_nirs)

  #Resampling as per original dimensions
  nirs_data = DownSampleNIRS(nirs_data, 25)      # 25 was the shape of original nirs data for MI (10 Hz)

  #Transforming Synthetic Labels
  synthetic_labels = np.round(synthetic_labels)
  synthetic_labels = np.concatenate(synthetic_labels, axis=0)

In [ ]:
if type == 'wg':

  #Reading the Corresponding generator and discriminator
  generator = load_model("/content/drive/MyDrive/generator_wg.h5", custom_objects=custom_objects)
  discriminator = load_model("/content/drive/MyDrive/discriminator_wg.h5", custom_objects=custom_objects)

  #These values below have been taken from the original data
  min_eeg, max_eeg, min_nirs, max_nirs = -2128.3276752716206, 3285.4588652976886, -0.11055875921617347, 0.11881306590112062

  #Generating Data
  num_samples = 1000
  latent_dim = 100
  synthetic_data, synthetic_labels = generate_synthetic_data(generator, discriminator, num_samples, latent_dim)

  #Extracting EEG Data
  eeg_data = synthetic_data[:,:,0:30]

  #Extracting NIRS data
  nirs_data = synthetic_data[:,:,30:]
  nirs_data = NIRS_ReScaling(nirs_data, min_eeg, max_eeg, min_nirs, max_nirs)

  #Resampling as per original dimensions
  nirs_data = DownSampleNIRS(nirs_data, 270)      # 270 was the shape of original nirs data for MI (10 Hz)

  #Transforming Synthetic Labels
  synthetic_labels = np.round(synthetic_labels)
  synthetic_labels = np.concatenate(synthetic_labels, axis=0)

In [ ]:
if type == 'wg':

  #Reading the Corresponding generator and discriminator
  generator = load_model("/content/drive/MyDrive/generator_wg.h5", custom_objects=custom_objects)
  discriminator = load_model("/content/drive/MyDrive/discriminator_wg.h5", custom_objects=custom_objects)

  #These values below have been taken from the original data
  min_eeg, max_eeg, min_nirs, max_nirs = -2128.3276752716206, 3285.4588652976886, -0.11055875921617347, 0.11881306590112062

  #Generating Data
  num_samples = 1000
  latent_dim = 100
  synthetic_data, synthetic_labels = generate_synthetic_data(generator, discriminator, num_samples, latent_dim)

  #Extracting EEG Data
  eeg_data = synthetic_data[:,:,0:30]

  #Extracting NIRS data
  nirs_data = synthetic_data[:,:,30:]
  nirs_data = NIRS_ReScaling(nirs_data, min_eeg, max_eeg, min_nirs, max_nirs)

  #Resampling as per original dimensions
  nirs_data = DownSampleNIRS(nirs_data, 270)      # 270 was the shape of original nirs data for MI (10 Hz)

  #Transforming Synthetic Labels
  threshold_1 = 0.3
  threshold_2 = 0.75

  # Apply thresholding
  synthetic_labels[synthetic_labels < threshold_1] = 0
  synthetic_labels[(synthetic_labels >= threshold_1) & (synthetic_labels < threshold_2)] = 1
  synthetic_labels[synthetic_labels >= threshold_2] = 2
  synthetic_labels = np.concatenate(synthetic_labels, axis=0)

In [ ]:
print('Minimum and Maximum Values of EEG and NIRS Data')
print(np.min(nirs_data))
print(np.max(nirs_data))
print(np.min(eeg_data))
print(np.max(eeg_data))

print('Shape of EEG and NIRS Data')
print(np.shape(nirs_data))
print(np.shape(eeg_data))

Minimum and Maximum Values of EEG and NIRS Data
-0.01800059899687767
1.3353936672210693
-2241.5933
3051.559
Shape of EEG and NIRS Data
(1000, 270, 72)
(1000, 5500, 32)
